# Use to Check Reproducibility - v2.1 Code

Version 2.1 relies on conda environment built from yaml file `fmda/install/fmda_ml.yml`. This environment uses python version `>=3.12` tensorflow version `>=2.16`. This led to substantial changes from the old reproducibility for code v2.0, so those old reproducibility hashes are no longer supported.

To see old code and reproduciblity results, see commit "911c6d7a" or PR#11.

## Setup

In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
from moisture_rnn import RNN, RNNParams, RNNData
import reproducibility
from utils import print_dict_summary, read_yml, read_pkl
from moisture_rnn_pkl import pkl2train
from moisture_rnn import RNN, RNNData, RNNParams
import logging
from utils import logging_setup
logging_setup()

### Reproducibility Dataset

In [ ]:
# Original File
repro_file='../data/reproducibility_dict_v2_TEST.pkl'
repro = read_pkl(repro_file)

## RNN with Stateful Batch Training


In [ ]:
# Set up params
params = repro['repro_info']['params']
print(type(params))
print(params)

In [ ]:
# Set up input data
rnn_dat = RNNData(repro, scaler = params['scaler'], features_list = params['features_list'])
rnn_dat.train_test_split(
    train_frac = params['train_frac'],
    val_frac = params['val_frac']
)
rnn_dat.scale_data()
rnn_dat.batch_reshape(timesteps = params['timesteps'], batch_size = params['batch_size'])

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat, reproducibility_run=True)

### Physics Initialized

In [ ]:
print("NOT YET IMPLEMENTED")
# params.update({'phys_initialize': True})
# reproducibility.set_seed()
# rnn = RNN(params)
# m, errs = rnn.run_model(rnn_dat)